# Resarch Notebook

In [36]:
import pandas as pd
FEATURE_DATA_FILE = "../data/featurized_race_data_v2.csv"
df = pd.read_csv(FEATURE_DATA_FILE)

# Remove some rows that have parsing errors
df = df[df["sex"].isin(['M','F'])]

# Enrich with some additional metadata
Add some additional metadata, like the BQ times and the AG relative time

In [45]:
bq_data = pd.read_csv("../data/boston_marathon_qualifying.csv")
import numpy as np
bq_data['min_age'] = bq_data['min_age'].apply(float)
# Keep only rows with valid age values
df["temp_join_idx"] = df.index
df_with_age = df[df['age'].notna()].copy()

# Then do the merge
df_with_age_sorted = df_with_age.sort_values('age')
df_with_age_sorted = pd.merge_asof(
    df_with_age_sorted,
    bq_data[['min_age', 'men_standard_minutes', 'women_standard_minutes', 'nonbinary_standard_minutes']],
    left_on='age',
    right_on='min_age',
    direction='backward'
)

# Add the BQ standard based on sex
conditions = [
    df_with_age_sorted['sex'].str.lower().isin(['m', 'male', 'man']),
    df_with_age_sorted['sex'].str.lower().isin(['f', 'female', 'woman'])
]
choices = [
    df_with_age_sorted['men_standard_minutes'],
    df_with_age_sorted['women_standard_minutes']
]
df_with_age_sorted['bq_standard_minutes'] = np.select(conditions, choices, default=df_with_age_sorted['nonbinary_standard_minutes'])

# If you need the full dataset back with nulls for missing ages:
df_with_age_sorted = df_with_age_sorted.drop(columns=['min_age', 'men_standard_minutes', 'women_standard_minutes', 'nonbinary_standard_minutes'])
df_enriched = df.merge(df_with_age_sorted[['temp_join_idx', 'bq_standard_minutes']], on='temp_join_idx', how='left')
df_enriched = df_enriched.drop(columns="temp_join_idx")

df_enriched["bq_adjusted_time"] = df_enriched.apply(
    lambda row:None if pd.isnull(row["bq_standard_minutes"]) else row["time"] - row["bq_standard_minutes"],
    axis=1)
df_enriched = df_enriched[df_enriched["bq_adjusted_time"].notna()]
display(df_enriched)

,age,sex,time,race,date,city,state,full_temp_min,full_temp_max,full_temp_median_min,...,full_overall_weekend_days_of_precip,peak_temp_min,peak_temp_max,peak_temp_median_min,peak_temp_median_max,peak_overall_precip,peak_overall_days_of_precip,peak_overall_weekend_days_of_precip,bq_standard_minutes,bq_adjusted_time
0,41.0,M,203.450000,"""last_chance_for_boston""_marathon",2003-02-02,ann arbor,mi,-4.3,61.5,21.95,...,8,-4.3,42.7,12.00,25.25,13.3,9,3,185.0,18.450000
1,48.0,M,203.366667,"""last_chance_for_boston""_marathon",2003-02-02,champaign,il,-1.0,63.9,25.75,...,10,-1.0,53.0,16.60,31.65,20.7,9,5,195.0,8.366667
2,26.0,F,220.983333,"""last_chance_for_boston""_marathon",2003-02-02,chicago,il,-3.5,62.4,26.90,...,12,-3.5,50.8,13.70,28.55,15.8,9,6,205.0,15.983333
3,43.0,M,194.733333,"""last_chance_for_boston""_marathon",2003-02-02,cincinnati,oh,-10.2,69.2,27.10,...,11,-10.2,52.6,19.05,30.90,15.9,10,5,185.0,9.733333
4,31.0,M,213.250000,"""last_chance_for_boston""_marathon",2003-02-02,cincinnati,oh,-10.2,69.2,27.10,...,11,-10.2,52.6,19.05,30.90,15.9,10,5,175.0,38.250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1687457,55.0,M,306.166667,zydeco_marathon,2020-03-08,fort worth,tx,26.1,78.3,40.60,...,9,26.1,78.3,42.00,60.65,112.8,12,3,210.0,96.166667
1687458,23.0,M,259.833333,zydeco_marathon,2020-03-08,houston,tx,31.9,82.1,47.25,...,11,34.1,77.7,49.70,67.70,110.5,15,4,175.0,84.833333
1687459,23.0,M,259.833333,zydeco_marathon,2020-03-08,houston,tx,31.9,82.1,47.25,...,11,34.1,77.7,49.70,67.70,110.5,15,4,175.0,84.833333
1687460,55.0,M,275.700000,zydeco_marathon,2020-03-08,mobile,al,29.6,75.6,48.20,...,13,35.7,73.5,49.45,65.40,127.4,15,3,210.0,65.700000


## Data Profiling

In [12]:
# Basic DataFrame Info
print(f"Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
print(f"\nMemory Usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print("\n" + "="*80)
df.info()

Shape: 1,688,270 rows × 21 columns

Memory Usage: 343.69 MB

<class 'pandas.DataFrame'>
RangeIndex: 1688270 entries, 0 to 1688269
Data columns (total 21 columns):
 #   Column                               Non-Null Count    Dtype  
---  ------                               --------------    -----  
 0   age                                  1447190 non-null  float64
 1   sex                                  1688255 non-null  str    
 2   time                                 1561622 non-null  float64
 3   race                                 1688270 non-null  str    
 4   date                                 1688270 non-null  str    
 5   city                                 1688270 non-null  str    
 6   state                                1688270 non-null  str    
 7   full_temp_min                        1688270 non-null  float64
 8   full_temp_max                        1688270 non-null  float64
 9   full_temp_median_min                 1688270 non-null  float64
 10  full_temp_median

In [13]:
# Categorical Columns Distribution
print("Categorical Columns Summary:")
print("="*80)

categorical_cols = df.select_dtypes(include=['object']).columns
if len(categorical_cols) > 0:
    print(f"Found {len(categorical_cols)} categorical columns\n")
    for col in categorical_cols:
        unique_count = df[col].nunique()
        print(f"\n{col}: {unique_count} unique values")
        if unique_count <= 20:  # Only show value counts for columns with <= 20 unique values
            print(df[col].value_counts().head(10))
        else:
            print(f"  Top 5 most common values:")
            print(df[col].value_counts().head(5))
else:
    print("No categorical columns found")

Categorical Columns Summary:
Found 5 categorical columns


sex: 356 unique values
  Top 5 most common values:
sex
M    947632
F    739890
X       189
m        54
f        18
Name: count, dtype: int64

race: 1278 unique values
  Top 5 most common values:
race
city_of_los_angeles_marathon_(l.a._marathon)    133561
chicago_marathon                                128585
marine_corps_marathon                            89408
new_york_city_marathon                           74800
boston_marathon                                  74144
Name: count, dtype: int64

date: 2625 unique values
  Top 5 most common values:
date
2016-10-09    17126
2012-10-07    16993
2011-10-09    16497
2023-11-05    16342
2014-10-12    14532
Name: count, dtype: int64

city: 85 unique values
  Top 5 most common values:
city
los angeles    121020
chicago        120839
new york        96250
houston         59373
san diego       57024
Name: count, dtype: int64

state: 33 unique values
  Top 5 most common values:
state
ca 

/var/folders/g1/1dxws3zs1djd1ytsd2h9g9wh0000gn/T/ipykernel_90620/2875196880.py:5: Pandas4Warning: For backward compatibility, 'str' dtypes are included by select_dtypes when 'object' dtype is specified. This behavior is deprecated and will be removed in a future version. Explicitly pass 'str' to `include` to select them, or to `exclude` to remove them and silence this warning.
See https://pandas.pydata.org/docs/user_guide/migration-3-strings.html#string-migration-select-dtypes for details on how to write code that works with pandas 2 and 3.
  categorical_cols = df.select_dtypes(include=['object']).columns


In [14]:
# Numeric Columns Distribution
print("Numeric Columns Summary:")
print("="*80)

numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
if len(numeric_cols) > 0:
    print(f"Found {len(numeric_cols)} numeric columns\n")
    for col in numeric_cols:
        print(f"\n{col}:")
        print(f"  Min: {df[col].min()}")
        print(f"  Max: {df[col].max()}")
        print(f"  Mean: {df[col].mean():.2f}")
        print(f"  Median: {df[col].median():.2f}")
        print(f"  Std Dev: {df[col].std():.2f}")
else:
    print("No numeric columns found")

Numeric Columns Summary:
Found 16 numeric columns


age:
  Min: 0.0
  Max: 3225.0
  Mean: 36.57
  Median: 35.00
  Std Dev: 11.29

time:
  Min: 122.71666666666668
  Max: 1299.75
  Mean: 284.07
  Median: 273.78
  Std Dev: 67.61

full_temp_min:
  Min: -36.9
  Max: 75.6
  Mean: 31.87
  Median: 33.80
  Std Dev: 16.97

full_temp_max:
  Min: 35.5
  Max: 118.5
  Mean: 87.40
  Median: 88.30
  Std Dev: 9.60

full_temp_median_min:
  Min: -1.5
  Max: 84.85
  Mean: 51.51
  Median: 52.80
  Std Dev: 13.02

full_temp_median_max:
  Min: 15.85
  Max: 108.0
  Mean: 68.76
  Median: 70.60
  Std Dev: 12.71

full_overall_precip:
  Min: 0.0
  Max: 1313.9
  Mean: 217.90
  Median: 212.30
  Std Dev: 138.44

full_overall_days_of_precip:
  Min: 0
  Max: 90
  Mean: 31.15
  Median: 33.00
  Std Dev: 14.07

full_overall_weekend_days_of_precip:
  Min: 0
  Max: 26
  Mean: 8.59
  Median: 9.00
  Std Dev: 4.19

peak_temp_min:
  Min: -36.9
  Max: 81.1
  Mean: 37.27
  Median: 37.10
  Std Dev: 13.54

peak_temp_max:
  Min: 25.

In [15]:
# Data Types and Unique Values
print("Column Data Types and Unique Values:")
print("="*80)

column_info = pd.DataFrame({
    'Column': df.columns,
    'Data_Type': df.dtypes.values,
    'Unique_Count': [df[col].nunique() for col in df.columns],
    'Sample_Values': [str(df[col].dropna().unique()[:3].tolist()) if df[col].nunique() <= 50 else 'Too many to display' for col in df.columns]
})

display(column_info)

Column Data Types and Unique Values:


,Column,Data_Type,Unique_Count,Sample_Values
0,age,float64,102,Too many to display
1,sex,str,356,Too many to display
2,time,float64,26974,Too many to display
3,race,str,1278,Too many to display
4,date,str,2625,Too many to display
5,city,str,85,Too many to display
6,state,str,33,"['mi', 'il', 'oh']"
7,full_temp_min,float64,995,Too many to display
8,full_temp_max,float64,768,Too many to display
9,full_temp_median_min,float64,2064,Too many to display


In [16]:
# Missing Values Analysis
print("Missing Values Analysis:")
print("="*80)

missing_data = pd.DataFrame({
    'Column': df.columns,
    'Missing_Count': df.isnull().sum().values,
    'Missing_Percent': (df.isnull().sum().values / len(df) * 100).round(2)
})

# Only show columns with missing values
missing_data = missing_data[missing_data['Missing_Count'] > 0].sort_values('Missing_Count', ascending=False)

if len(missing_data) > 0:
    display(missing_data)
else:
    print("No missing values found in any column!")

Missing Values Analysis:


,Column,Missing_Count,Missing_Percent
0,age,241080,14.28
2,time,126648,7.50
1,sex,15,0.00


In [17]:
# Statistical Summary
print("Statistical Summary (all columns):")
print("="*80)
display(df.describe(include='all').T)  # Transpose for better readability

Statistical Summary (all columns):


,count,unique,top,freq,mean,std,min,25%,50%,75%,max
age,1447190.0,NaN,NaN,NaN,36.574419,11.288382,0.0,28.0,35.0,44.0,3225.0
sex,1688255,356,M,947632,NaN,NaN,NaN,NaN,NaN,NaN,NaN
time,1561622.0,NaN,NaN,NaN,284.06795,67.605009,122.716667,235.5,273.783333,321.466667,1299.75
race,1688270,1278,city_of_los_angeles_marathon_(l.a._marathon),133561,NaN,NaN,NaN,NaN,NaN,NaN,NaN
date,1688270,2625,2016-10-09,17126,NaN,NaN,NaN,NaN,NaN,NaN,NaN
city,1688270,85,los angeles,121020,NaN,NaN,NaN,NaN,NaN,NaN,NaN
state,1688270,33,ca,362883,NaN,NaN,NaN,NaN,NaN,NaN,NaN
full_temp_min,1688270.0,NaN,NaN,NaN,31.868774,16.974256,-36.9,24.1,33.8,42.1,75.6
full_temp_max,1688270.0,NaN,NaN,NaN,87.395629,9.603565,35.5,81.5,88.3,94.1,118.5
full_temp_median_min,1688270.0,NaN,NaN,NaN,51.506585,13.018333,-1.5,42.2,52.8,61.85,84.85


In [18]:
# Data Preview
print("First 5 rows:")
print("="*80)
display(df.head())

print("\n\nLast 5 rows:")
print("="*80)
display(df.tail())

print("\n\nRandom sample of 5 rows:")
print("="*80)
display(df.sample(5))

First 5 rows:


,age,sex,time,race,date,city,state,full_temp_min,full_temp_max,full_temp_median_min,...,full_overall_precip,full_overall_days_of_precip,full_overall_weekend_days_of_precip,peak_temp_min,peak_temp_max,peak_temp_median_min,peak_temp_median_max,peak_overall_precip,peak_overall_days_of_precip,peak_overall_weekend_days_of_precip
0,41.0,M,203.450000,"""last_chance_for_boston""_marathon",2003-02-02,ann arbor,mi,-4.3,61.5,21.95,...,104.1,32,8,-4.3,42.7,12.00,25.25,13.3,9,3
1,48.0,M,203.366667,"""last_chance_for_boston""_marathon",2003-02-02,champaign,il,-1.0,63.9,25.75,...,117.7,29,10,-1.0,53.0,16.60,31.65,20.7,9,5
2,26.0,F,220.983333,"""last_chance_for_boston""_marathon",2003-02-02,chicago,il,-3.5,62.4,26.90,...,94.2,30,12,-3.5,50.8,13.70,28.55,15.8,9,6
3,43.0,M,194.733333,"""last_chance_for_boston""_marathon",2003-02-02,cincinnati,oh,-10.2,69.2,27.10,...,210.7,37,11,-10.2,52.6,19.05,30.90,15.9,10,5
4,31.0,M,213.250000,"""last_chance_for_boston""_marathon",2003-02-02,cincinnati,oh,-10.2,69.2,27.10,...,210.7,37,11,-10.2,52.6,19.05,30.90,15.9,10,5




Last 5 rows:


,age,sex,time,race,date,city,state,full_temp_min,full_temp_max,full_temp_median_min,...,full_overall_precip,full_overall_days_of_precip,full_overall_weekend_days_of_precip,peak_temp_min,peak_temp_max,peak_temp_median_min,peak_temp_median_max,peak_overall_precip,peak_overall_days_of_precip,peak_overall_weekend_days_of_precip
1688265,64.0,M,NaN,zydeco_marathon,2023-03-12,phoenix,az,32.0,76.8,42.55,...,161.7,22,4,33.3,76.8,43.70,64.35,56.0,8,2
1688266,64.0,M,NaN,zydeco_marathon,2023-03-12,phoenix,az,32.0,76.8,42.55,...,161.7,22,4,33.3,76.8,43.70,64.35,56.0,8,2
1688267,27.0,F,NaN,zydeco_marathon,2023-03-12,shreveport,la,11.1,81.4,43.35,...,398.7,45,10,30.4,81.4,51.80,73.25,90.5,18,4
1688268,43.0,M,NaN,zydeco_marathon,2023-03-12,shreveport,la,11.1,81.4,43.35,...,398.7,45,10,30.4,81.4,51.80,73.25,90.5,18,4
1688269,59.0,F,NaN,zydeco_marathon,2023-03-12,tulsa,ok,2.1,74.2,34.30,...,205.8,31,5,22.8,74.2,41.25,61.35,94.5,14,3




Random sample of 5 rows:


,age,sex,time,race,date,city,state,full_temp_min,full_temp_max,full_temp_median_min,...,full_overall_precip,full_overall_days_of_precip,full_overall_weekend_days_of_precip,peak_temp_min,peak_temp_max,peak_temp_median_min,peak_temp_median_max,peak_overall_precip,peak_overall_days_of_precip,peak_overall_weekend_days_of_precip
785638,34.0,F,291.433333,houston_marathon,2000-01-16,houston,tx,31.9,82.4,51.10,...,158.6,22,6,31.9,77.8,47.45,65.55,60.0,9,2
1259665,24.0,M,263.483333,philadelphia_marathon,2019-11-24,pittsburgh,pa,11.1,90.1,49.45,...,258.9,34,10,11.1,70.2,32.05,48.30,95.0,12,3
909116,NaN,M,223.916667,madison_marathon_(mad_city_marathon),2003-05-25,champaign,il,2.5,85.4,43.35,...,209.2,41,11,37.4,85.4,51.90,69.40,125.3,19,5
704261,NaN,M,217.233333,green_bay_marathon,2003-05-18,milwaukee,wi,-3.7,71.8,32.90,...,202.4,44,10,28.7,68.7,41.25,52.90,113.3,14,5
465001,28.0,M,234.100000,city_of_los_angeles_marathon_(l.a._marathon),2015-03-15,oakland,ca,32.7,79.1,45.60,...,122.8,12,3,41.2,79.1,47.05,65.70,2.2,2,1


## Location Performance Questions
1. Which major cities have the fastest runners (overall and median)?

In [17]:
# Question 1: Which major cities have the fastest runners (overall and median)?

# Filter to major cities (cities with at least 10,000 runners)
city_counts = df.groupby(['city', 'state']).size()
major_cities = city_counts[city_counts >= 1000].index

# Filter df to only major cities
df_major = df[df.apply(lambda row: (row['city'], row['state']) in major_cities, axis=1)]

print(f"Analyzing {len(major_cities)} major cities (10,000+ runners each)")
print("="*80)

# Convert time to seconds for easier analysis
def time_to_seconds(time_minutes:int):
    return time_minutes*60

# Apply conversion
df_major['time_seconds'] = df_major['time'].apply(time_to_seconds)

# Group by city and calculate statistics
city_performance = df_major.groupby(['city', 'state'])['time_seconds'].agg([
    ('count', 'count'),
    ('mean_seconds', 'mean'),
    ('median_seconds', 'median'),
    ('std_seconds', 'std')
]).reset_index()

# Convert back to time format for display
def seconds_to_time(seconds):
    """Convert seconds to HH:MM:SS format"""
    if pd.isna(seconds):
        return None
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    secs = int(seconds % 60)
    return f"{hours:02d}:{minutes:02d}:{secs:02d}"

city_performance['mean_time'] = city_performance['mean_seconds'].apply(seconds_to_time)
city_performance['median_time'] = city_performance['median_seconds'].apply(seconds_to_time)

# Sort by fastest mean time
#city_performance_sorted_mean = city_performance.sort_values('mean_seconds').head(20)
#print("\n\nTop 20 Fastest Cities by MEAN finish time:")
#print("="*80)
#display(city_performance_sorted_mean[['city', 'state', 'count', 'mean_time', 'median_time']])

# Sort by fastest median time
city_performance_sorted_median = city_performance.sort_values('median_seconds').head(10)
print("\n\nTop 10 Fastest Cities by MEDIAN finish time:")
print("="*80)
display(city_performance_sorted_median[['city', 'state', 'count', 'mean_time', 'median_time']])

# Find slowest cities for comparison
city_performance_sorted_slowest = city_performance.sort_values('median_seconds', ascending=False).head(10)
print("\n\nSlowest 10 Major Cities by MEDIAN finish time (for comparison):")
print("="*80)
display(city_performance_sorted_slowest[['city', 'state', 'count', 'mean_time', 'median_time']])

# NEW ANALYSIS: Top 10% of runners by city
print("\n\n" + "="*80)
print("ELITE RUNNER ANALYSIS - Top 10% of Runners by City")
print("="*80)

# Calculate top 10% performance for each major city
elite_performance = []

for city, state in major_cities:
    city_df = df_major[(df_major['city'] == city) & (df_major['state'] == state)]
    city_df_valid = city_df[city_df['time_seconds'].notna()]
    
    if len(city_df_valid) > 0:
        # Get top 10% (fastest times = lowest seconds)
        top_10_pct_threshold = city_df_valid['time_seconds'].quantile(0.10)
        top_10_pct = city_df_valid[city_df_valid['time_seconds'] <= top_10_pct_threshold]
        
        elite_performance.append({
            'city': city,
            'state': state,
            'total_runners': len(city_df_valid),
            'elite_count': len(top_10_pct),
            'elite_mean_seconds': top_10_pct['time_seconds'].mean(),
            'elite_median_seconds': top_10_pct['time_seconds'].median(),
            'elite_max_seconds': top_10_pct['time_seconds'].max(),  # Slowest of the elite
            'elite_min_seconds': top_10_pct['time_seconds'].min()   # Fastest overall
        })

elite_df = pd.DataFrame(elite_performance)

# Convert times to readable format
elite_df['elite_mean_time'] = elite_df['elite_mean_seconds'].apply(seconds_to_time)
elite_df['elite_median_time'] = elite_df['elite_median_seconds'].apply(seconds_to_time)
elite_df['elite_max_time'] = elite_df['elite_max_seconds'].apply(seconds_to_time)
elite_df['elite_min_time'] = elite_df['elite_min_seconds'].apply(seconds_to_time)

# Sort by fastest elite mean time
elite_sorted = elite_df.sort_values('elite_mean_seconds')

print("\n\nTop 10 Cities with Fastest ELITE Runners (top 10% by median):")
print("="*80)
elite_sorted_median = elite_df.sort_values('elite_median_seconds')
display(elite_sorted_median.head(10)[['city', 'state', 'total_runners', 'elite_count', 
                                        'elite_mean_time', 'elite_median_time', 'elite_min_time']])

# Show cities with the absolute fastest individual runners
print("\n\nCities with the Absolute Fastest Individual Runners:")
print("="*80)
elite_fastest_individual = elite_df.sort_values('elite_min_seconds')
display(elite_fastest_individual.head(10)[['city', 'state', 'total_runners', 
                                             'elite_min_time', 'elite_mean_time', 'elite_median_time']])

Analyzing 85 major cities (10,000+ runners each)


Top 10 Fastest Cities by MEDIAN finish time:


,city,state,count,mean_time,median_time
14,cambridge,ma,10499,04:09:22,04:01:24
48,new york,ny,62877,04:19:21,04:10:54
11,boulder,co,11027,04:20:14,04:12:09
40,madison,wi,13504,04:19:57,04:13:11
44,minneapolis,mn,38260,04:20:21,04:13:43
12,brooklyn,ny,24883,04:23:10,04:14:11
10,boston,ma,20957,04:18:33,04:14:12
3,ann arbor,mi,10318,04:22:39,04:15:05
52,omaha,ne,11156,04:24:25,04:16:37
20,columbus,oh,21788,04:25:33,04:17:12




Slowest 10 Major Cities by MEDIAN finish time (for comparison):


,city,state,count,mean_time,median_time
31,honolulu,hi,40656,05:51:26,05:34:44
37,los angeles,ca,118834,05:29:30,05:24:29
62,riverside,ca,7418,05:21:46,05:12:12
75,south pasadena,ca,1817,05:14:50,05:08:14
69,santa clarita,ca,2136,05:13:33,05:04:00
36,long beach,ca,17201,05:10:43,05:01:15
53,pasadena,ca,9850,05:07:00,04:58:37
80,tustin,ca,2901,05:03:40,04:56:20
70,santa monica,ca,12649,04:58:42,04:48:51
26,fort worth,tx,10415,04:54:19,04:47:47




ELITE RUNNER ANALYSIS - Top 10% of Runners by City


Top 10 Cities with Fastest ELITE Runners (top 10% by median):


,city,state,total_runners,elite_count,elite_mean_time,elite_median_time,elite_min_time
11,boulder,co,11027,1103,02:47:40,02:51:56,02:11:53
14,cambridge,ma,10499,1050,02:53:19,02:56:24,02:10:17
10,boston,ma,20957,2098,03:00:19,03:02:27,02:08:26
48,new york,ny,62877,6290,03:01:24,03:03:53,02:11:05
0,albuquerque,nm,11787,1180,02:59:42,03:04:10,02:11:41
40,madison,wi,13504,1352,03:02:10,03:04:39,02:11:39
12,brooklyn,ny,24883,2489,03:02:27,03:04:41,02:14:58
84,westminster,co,2098,210,03:03:57,03:06:02,02:16:31
59,provo,ut,7721,773,03:02:35,03:06:24,02:12:56
44,minneapolis,mn,38260,3829,03:03:50,03:06:31,02:12:24




Cities with the Absolute Fastest Individual Runners:


,city,state,total_runners,elite_min_time,elite_mean_time,elite_median_time
77,tampa,fl,13827,02:02:43,03:17:57,03:21:57
10,boston,ma,20957,02:08:26,03:00:19,03:02:27
66,san diego,ca,52457,02:08:37,03:14:37,03:17:51
16,charlotte,nc,22740,02:09:06,03:07:32,03:10:16
38,louisville,ky,16187,02:09:19,03:12:19,03:15:56
57,portland,or,43917,02:09:20,03:07:01,03:10:00
78,tucson,az,13873,02:09:47,03:09:04,03:13:07
34,irvine,ca,12036,02:09:56,03:17:40,03:20:31
37,los angeles,ca,118834,02:09:58,03:29:06,03:33:06
14,cambridge,ma,10499,02:10:17,02:53:19,02:56:24


2. What is the relation of weather during the training period (full and peak) to the performance during the race?  How do things like heat, cold, and precipitation effect performance?

In [4]:
# Question 2: Relation of weather during training period to race performance

# First, check what weather columns we have
weather_cols = [col for col in df.columns if 'weather' in col.lower() or 'temp' in col.lower() or 'precip' in col.lower()]
print("Available weather-related columns:")
print(weather_cols)
print("="*80)

# If we have weather columns, analyze their relationship with performance
if len(weather_cols) > 0:
    # Create a copy with time in seconds
    df_weather = df.copy()
    df_weather['time_seconds'] = df_weather['time'].apply(time_to_seconds)
    
    # Remove invalid times
    df_weather = df_weather[df_weather['time_seconds'].notna()]
    
    # For each weather column, calculate correlation with performance
    print("\n\nCorrelation between weather features and finish time:")
    print("(Positive correlation = worse performance with higher values)")
    print("="*80)
    
    weather_correlations = {}
    for col in weather_cols:
        if df_weather[col].dtype in ['float64', 'int64']:
            corr = df_weather[[col, 'time_seconds']].corr().iloc[0, 1]
            weather_correlations[col] = corr
            print(f"{col}: {corr:.4f}")
    
    # Analyze specific weather patterns
    print("\n\nAnalyzing temperature effects on performance:")
    print("="*80)
    
    # Check for temperature columns (full and peak training periods)
    temp_cols = [col for col in weather_cols if 'temp' in col.lower()]
    
    for temp_col in temp_cols:
        if df_weather[temp_col].dtype in ['float64', 'int64']:
            # Bin temperatures and calculate average finish times
            df_weather[f'{temp_col}_bin'] = pd.cut(df_weather[temp_col], bins=10)
            temp_performance = df_weather.groupby(f'{temp_col}_bin')['time_seconds'].agg(['mean', 'median', 'count'])
            temp_performance['mean_time'] = temp_performance['mean'].apply(seconds_to_time)
            temp_performance['median_time'] = temp_performance['median'].apply(seconds_to_time)
            
            print(f"\n{temp_col} effect on performance:")
            display(temp_performance[['count', 'mean_time', 'median_time']])
    
    # Analyze precipitation effects
    print("\n\nAnalyzing precipitation effects on performance:")
    print("="*80)
    
    precip_cols = [col for col in weather_cols if 'precip' in col.lower()]
    
    for precip_col in precip_cols:
        if df_weather[precip_col].dtype in ['float64', 'int64']:
            # Categorize precipitation levels
            df_weather[f'{precip_col}_category'] = pd.cut(
                df_weather[precip_col], 
                bins=[-0.1, 0.1, 1, 5, 100],
                labels=['Dry (<0.1)', 'Light (0.1-1)', 'Moderate (1-5)', 'Heavy (>5)']
            )
            
            precip_performance = df_weather.groupby(f'{precip_col}_category')['time_seconds'].agg(['mean', 'median', 'count'])
            precip_performance['mean_time'] = precip_performance['mean'].apply(seconds_to_time)
            precip_performance['median_time'] = precip_performance['median'].apply(seconds_to_time)
            
            print(f"\n{precip_col} effect on performance:")
            display(precip_performance[['count', 'mean_time', 'median_time']])
            
else:
    print("\nNo weather columns found in the dataset.")
    print("Weather features may need to be joined with the race data first.")

Available weather-related columns:
['full_temp_min', 'full_temp_max', 'full_temp_median_min', 'full_temp_median_max', 'full_overall_precip', 'full_overall_days_of_precip', 'full_overall_weekend_days_of_precip', 'peak_temp_min', 'peak_temp_max', 'peak_temp_median_min', 'peak_temp_median_max', 'peak_overall_precip', 'peak_overall_days_of_precip', 'peak_overall_weekend_days_of_precip']


Correlation between weather features and finish time:
(Positive correlation = worse performance with higher values)
full_temp_min: 0.1302
full_temp_max: 0.0089
full_temp_median_min: 0.0620
full_temp_median_max: 0.0650
full_overall_precip: -0.0811
full_overall_days_of_precip: -0.0931
full_overall_weekend_days_of_precip: -0.0661
peak_temp_min: 0.1088
peak_temp_max: 0.0458
peak_temp_median_min: 0.0843
peak_temp_median_max: 0.0755
peak_overall_precip: -0.0577
peak_overall_days_of_precip: -0.0714
peak_overall_weekend_days_of_precip: -0.0330


Analyzing temperature effects on performance:

full_temp_min effect 

,count,mean_time,median_time
full_temp_min_bin,,,
"(-37.012, -25.65]",1130,04:38:23,04:21:52
"(-25.65, -14.4]",18286,04:28:26,04:16:34
"(-14.4, -3.15]",54847,04:25:24,04:15:42
"(-3.15, 8.1]",88707,04:28:15,04:18:52
"(8.1, 19.35]",143940,04:35:54,04:26:15
"(19.35, 30.6]",312268,04:40:17,04:30:19
"(30.6, 41.85]",543414,04:48:24,04:38:27
"(41.85, 53.1]",304242,04:43:41,04:35:50
"(53.1, 64.35]",48903,04:44:27,04:36:00



full_temp_max effect on performance:


,count,mean_time,median_time
full_temp_max_bin,,,
"(35.417, 43.8]",760,04:27:16,04:17:51
"(43.8, 52.1]",1598,04:38:08,04:27:01
"(52.1, 60.4]",11083,04:33:28,04:22:15
"(60.4, 68.7]",50169,04:30:14,04:19:00
"(68.7, 77.0]",151758,04:37:21,04:25:55
"(77.0, 85.3]",419988,04:53:31,04:40:32
"(85.3, 93.6]",496357,04:41:09,04:32:19
"(93.6, 101.9]",364728,04:41:30,04:33:18
"(101.9, 110.2]",58863,04:47:37,04:38:54



full_temp_median_min effect on performance:


...

,count,mean_time,median_time
peak_overall_precip_category,,,
Dry (<0.1),42561,04:55:56,04:47:16
Light (0.1-1),31748,04:47:27,04:39:11
Moderate (1-5),55583,04:47:37,04:38:22
Heavy (>5),1014637,04:45:46,04:35:04



peak_overall_days_of_precip effect on performance:


,count,mean_time,median_time
peak_overall_days_of_precip_category,,,
Dry (<0.1),52086,04:54:05,04:45:22
Light (0.1-1),55964,04:53:06,04:44:13
Moderate (1-5),198324,04:52:15,04:41:46
Heavy (>5),1255248,04:41:57,04:31:42



peak_overall_weekend_days_of_precip effect on performance:


,count,mean_time,median_time
peak_overall_weekend_days_of_precip_category,,,
Dry (<0.1),179158,04:48:51,04:39:32
Light (0.1-1),212168,04:46:15,04:36:36
Moderate (1-5),1021409,04:43:30,04:33:00
Heavy (>5),148887,04:38:59,04:28:27


3. Are there differences in how training weather effect performance based on location, age, or sex?  Are there places that are "hardier" (i.e. training weather has less of an effect)?  Are older people more or less able to manage bad weather in training?  Etc.

In [14]:
# Question 3: Differences in weather effects by location, age, and sex

# Check for demographic columns
demo_cols = [col for col in df.columns if any(x in col.lower() for x in ['age', 'sex', 'gender'])]
print("Available demographic columns:")
print(demo_cols)
print("="*80)

weather_cols = [col for col in df.columns if 'weather' in col.lower() or 'temp' in col.lower() or 'precip' in col.lower()]

if len(weather_cols) > 0 and len(demo_cols) > 0:
    
    df_interact = df.copy()
    df_interact['time_seconds'] = df_interact['time'].apply(time_to_seconds)
    df_interact = df_interact[df_interact['time_seconds'].notna()]
    
    # Analyze by sex/gender if available
    sex_cols = [col for col in demo_cols if 'sex' in col.lower() or 'gender' in col.lower()]
    if len(sex_cols) > 0:
        sex_col = sex_cols[0]
        print(f"\n\nAnalyzing weather effects by {sex_col}:")
        print("="*80)
        
        # For a key weather variable, see if effects differ by sex
        if len(weather_cols) > 0:
            # Use first temperature column if available
            temp_cols = [col for col in weather_cols if 'temp' in col.lower() and 'median' in col.lower()]
            if len(temp_cols) > 0:
                temp_col = temp_cols[0]
                
                # Create temperature categories
                df_interact[f'{temp_col}_category'] = pd.cut(
                    df_interact[temp_col],
                    bins=3,
                    labels=['Cold', 'Moderate', 'Hot']
                )
                
                print(f"\nPerformance by {temp_col} and {sex_col}:")
                sex_temp_performance = df_interact.groupby([sex_col, f'{temp_col}_category'])['time_seconds'].agg(['mean', 'count'])
                sex_temp_performance['mean_time'] = sex_temp_performance['mean'].apply(seconds_to_time)
                display(sex_temp_performance[['count', 'mean_time']])
    
    # Analyze by age if available
    age_cols = [col for col in demo_cols if 'age' in col.lower()]
    if len(age_cols) > 0:
        age_col = age_cols[0]
        print(f"\n\nAnalyzing weather effects by {age_col}:")
        print("="*80)
        
        # Create age categories
        df_interact['age_category'] = pd.cut(
            df_interact[age_col],
            bins=[0, 30, 40, 50, 60, 100],
            labels=['Under 30', '30-39', '40-49', '50-59', '60+']
        )
        
        if len(weather_cols) > 0:
            temp_cols = [col for col in weather_cols if 'temp' in col.lower() and 'median' in col.lower()]
            if len(temp_cols) > 0:
                temp_col = temp_cols[0]
                
                df_interact[f'{temp_col}_category'] = pd.cut(
                    df_interact[temp_col],
                    bins=3,
                    labels=['Cold', 'Moderate', 'Hot']
                )
                
                print(f"\nPerformance by {temp_col} and age category:")
                age_temp_performance = df_interact.groupby(['age_category', f'{temp_col}_category'])['time_seconds'].agg(['mean', 'count'])
                age_temp_performance['mean_time'] = age_temp_performance['mean'].apply(seconds_to_time)
                display(age_temp_performance[['count', 'mean_time']])
    
    # Analyze "hardiness" by location
    print("\n\nAnalyzing location 'hardiness' (cities where weather has less effect):")
    print("="*80)
    
    # For major cities, calculate how much performance varies with weather
    major_cities = df_interact.groupby(['city', 'state']).size()
    major_cities = major_cities[major_cities >= 5000].index
    
    if len(weather_cols) > 0:
        temp_cols = [col for col in weather_cols if 'temp' in col.lower() and 'median' in col.lower()]
        if len(temp_cols) > 0:
            temp_col = temp_cols[0]
            
            # For each major city, calculate correlation between weather and performance
            city_weather_sensitivity = {}
            
            for city, state in major_cities:
                city_df = df_interact[(df_interact['city'] == city) & (df_interact['state'] == state)]
                if len(city_df) > 100 and city_df[temp_col].notna().sum() > 100:
                    corr = city_df[[temp_col, 'time_seconds']].corr().iloc[0, 1]
                    city_weather_sensitivity[(city, state)] = {
                        'correlation': corr,
                        'count': len(city_df)
                    }
            
            # Convert to dataframe
            sensitivity_df = pd.DataFrame([
                {'city': city, 'state': state, 'temp_correlation': data['correlation'], 'count': data['count']}
                for (city, state), data in city_weather_sensitivity.items()
            ])
            
            # Sort by correlation (lower = more hardy, less affected by temperature)
            sensitivity_df = sensitivity_df.sort_values('temp_correlation')
            
            print(f"\nMost 'hardy' cities (least affected by {temp_col}):")
            print("(Lower correlation = better performance despite temperature variation)")
            display(sensitivity_df.head(15))
            
            print(f"\nLeast 'hardy' cities (most affected by {temp_col}):")
            print("(Higher correlation = worse performance with temperature variation)")
            display(sensitivity_df.tail(15))
            
else:
    print("\nInsufficient weather or demographic data for interaction analysis.")

Available demographic columns:
['age', 'sex']


Analyzing weather effects by sex:

Performance by full_temp_median_min and sex:


count mean_time
sex full_temp_median_min_category                  
F   Cold                            26260  04:38:06
    Moderate                       384981  05:02:22
    Hot                            270355  04:59:39
M   Cold                            33255  04:14:23
    Moderate                       507578  04:31:41
    Hot                            338527  04:32:42



Analyzing weather effects by age:

Performance by full_temp_median_min and age category:


count mean_time
age_category full_temp_median_min_category                  
Under 30     Cold                            18461  04:20:22
             Moderate                       273127  04:46:09
             Hot                            181725  04:35:58
30-39        Cold                            16258  04:18:59
             Moderate                       251323  04:37:28
             Hot                            176282  04:37:14
40-49        Cold                             9891  04:26:40
             Moderate                       159534  04:44:27
             Hot                            107058  04:45:06
50-59        Cold                             4741  04:43:48
             Moderate                        72835  05:02:14
             Hot                             48592  05:02:14
60+          Cold                             1361  05:07:38
             Moderate                        21472  05:32:36
             Hot                             14704  05:31:06



Analyzing location 'hardiness' (cities where weather has less effect):

Most 'hardy' cities (least affected by full_temp_median_min):
(Lower correlation = better performance despite temperature variation)


,city,state,temp_correlation,count
0,albuquerque,nm,-0.269713,11787
27,los angeles,ca,-0.162472,118834
45,riverside,ca,-0.150718,7418
38,pasadena,ca,-0.140811,9850
52,santa monica,ca,-0.120343,12649
26,long beach,ca,-0.109255,17201
24,irvine,ca,-0.090472,12036
25,jacksonville,fl,-0.067352,12215
8,boston,ma,-0.063109,20957
41,pittsburgh,pa,-0.062322,27794



Least 'hardy' cities (most affected by full_temp_median_min):
(Higher correlation = worse performance with temperature variation)


,city,state,temp_correlation,count
59,virginia beach,va,0.070830,11167
18,denver,co,0.072008,30752
51,san jose,ca,0.073107,22796
23,houston,tx,0.079183,55577
1,alexandria,va,0.082453,24371
13,chicago,il,0.111947,117314
4,arlington,va,0.118070,31318
19,fairfax,va,0.118304,8020
60,washington,dc,0.126345,47300
20,falls church,va,0.146723,5237
